### Melon Top100 chart
- 100곡의 노래 제목과 ID 추출 후, 저장
- 100곡의 노래의 상세 정보를 추출해서 list와 dict에 저장 후, json 파일로 저장
- json 파일을 load하여 Pandas의 DataFrame에 저장
- DataFrame 객체를 DB의 table에 저장
#### 정규 표현식
- https://www.nextree.co.kr/p4327/

In [2]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
req_header_dict = {
    # 요청헤더(브라우저 정보)
   'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}
res = requests.get(url, headers=req_header_dict)
# print(res.status_code)

if(res.ok):
    html = res.text # url의 elments 모든 text 가져오기
    # BeautifulSoup 생성자에서 html을 파싱하고 그 결과를 BeautifulSoup 객체로 반환한다.
    soup = BeautifulSoup(html, 'html.parser')
    # playSong이 들어간 a href태그(div태그의 id가 tb_list인 tr안에 있음)
    print(len(soup.select("div#tb_list tr a[href*='playSong']")))
    a_tags = soup.select("div#tb_list tr a[href*='playSong']")
    
    song_list=[]
    for idx, a_tag in enumerate(a_tags,1):
        song_title = a_tag.text # a 태그안에 text요소(노래 제목) 가져오기
        song_dict={}
        song_dict['title'] = song_title
        href_value = a_tag['href']  # a 태그안에 href요소 가져오기
        # print(href_value)
        # 정규표현식 사용(re)해서 노래 ID찾기
        matched = re.search(r'(\d+)\);', href_value)
        # print(matched.group(1))
        if matched:
            song_id = matched.group(1)
            song_dict['id'] = song_id
            song_detail_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
            song_dict['url'] = song_detail_url
            # print(idx, song_title, song_id, song_detail_url)
            print(song_dict)
            song_list.append(song_dict)
print(len(song_list))

100
{'title': '사랑은 늘 도망가', 'id': '34061322', 'url': 'https://www.melon.com/song/detail.htm?songId=34061322'}
{'title': '취중고백', 'id': '34431086', 'url': 'https://www.melon.com/song/detail.htm?songId=34431086'}
{'title': '호랑수월가', 'id': '34535898', 'url': 'https://www.melon.com/song/detail.htm?songId=34535898'}
{'title': 'Step Back', 'id': '34494149', 'url': 'https://www.melon.com/song/detail.htm?songId=34494149'}
{'title': '회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)', 'id': '34256568', 'url': 'https://www.melon.com/song/detail.htm?songId=34256568'}
{'title': 'ELEVEN', 'id': '34349913', 'url': 'https://www.melon.com/song/detail.htm?songId=34349913'}
{'title': '리무진 (Feat. MINO) (Prod. GRAY)', 'id': '34298499', 'url': 'https://www.melon.com/song/detail.htm?songId=34298499'}
{'title': 'Counting Stars (Feat. Beenzino)', 'id': '34398735', 'url': 'https://www.melon.com/song/detail.htm?songId=34398735'}
{'title': 'Dreams Come True', 'id': '34436674', 'url': 'https://www.melon.com/song/detail.htm?son

In [38]:
import requests
from bs4 import BeautifulSoup
import re

req_header_dict = {
    # 요청헤더(브라우저 정보)
   'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}

song_detail_list = []
for idx,song in enumerate(song_list,1):
    song_detail_dict = {} # 노래 1곡의 상세정보를 저장하는 dict
    song_detail_url = song['url']
    
    res = requests.get(song_detail_url, headers=req_header_dict)    
    #print(res.status_code)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        print(idx, song['title'])
        song_detail_dict['곡명'] = song['title']
        singer_span = soup.select("a[href*='goArtistDetail'] span")
        if singer_span:
            song_detail_dict['가수'] = singer_span[0].text
        
        #print(soup.select("div.meta dd"))
        song_dd = soup.select("div.meta dd")
        if song_dd:
            # print(song_dd[0].text)
            song_detail_dict['앨범'] = song_dd[0].text
            song_detail_dict['발매일'] = song_dd[1].text
            song_detail_dict['장르'] = song_dd[2].text  
        
        song_id = song['id']
        like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
        like_res = requests.get(like_url, headers=req_header_dict)
        if like_res.ok:
            #print(like_res.json())
            song_detail_dict['좋아요'] = like_res.json()['contsLike'][0]['SUMMCNT']
        
        lyric_div = soup.select("div#d_video_summary")
        if lyric_div:
            lyric_temp = lyric_div[0].text
            pattern = re.compile(r'[\r\n\t]')
            # \t\n\t 특수문자를 ''(empty string)으로 대체해라
            lyric = pattern.sub('', lyric_temp.strip())
        else:
            lyric = ''
        song_detail_dict['가사'] = lyric
        
        print(song_detail_dict)
        song_detail_list.append(song_detail_dict)
print(len(song_detail_list))
#song_detail_list[:5]

1 사랑은 늘 도망가
{'곡명': '사랑은 늘 도망가', '가수': '임영웅', '앨범': '신사와 아가씨 OST Part.2', '발매일': '2021.10.11', '장르': '발라드, 국내드라마', '좋아요': 123988, '가사': '눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨었지내 아름답던 사람아사랑이란 게 참 쓰린 거더라잡으려 할수록 더 멀어지더라이별이란 게 참 쉬운 거더라내 잊지 못할 사람아사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데바람이 분다 옷깃을 세워도차가운 이별의 눈물이 차올라잊지 못해서 가슴에 사무친내 소중했던 사람아사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데기다림도 애태움도 다 버려야 하는데무얼 찾아 이 길을 서성일까무얼 찾아 여기 있나사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데잠시 쉬어가면 좋을 텐데'}
2 취중고백
{'곡명': '취중고백', '가수': '김민석 (멜로망스)', '앨범': '취중고백', '발매일': '2021.12.19', '장르': '발라드', '좋아요': 68686, '가사': '뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서 오랜만에술을 좀 했는데자꾸만 니 얼굴 떠올라무작정 달려왔어이 맘 모르겠니요즘 난 미친 사람처럼너만 생각해대책없이 네가 점점 좋아져아냐 안 취했어 진짜야널 정말 사랑해눈물이 날만큼 원하고 있어정말로 몰랐니가끔 전화해 장난치듯주말엔 뭐할거냐며너의 관심 끌던 나를그리고 한번씩 누나 주려 샀는데너 그냥 준다고생색 낸 선물도 너 때문에 산거야이 맘 모르겠니요즘 난 미친 사람처럼너만 생각해대책없이 네가 점점 좋아져아냐 안 취했어 진짜야널 정말 사랑해진심이야 믿어줘갑자기 이런 말 놀랐다면 미안해부담이 되는게 당연해이해해

10 눈이 오잖아(Feat.헤이즈)
{'곡명': '눈이 오잖아(Feat.헤이즈)', '가수': '이무진', '앨범': '눈이 오잖아(Feat.헤이즈)', '발매일': '2021.12.03', '장르': '발라드', '좋아요': 61577, '가사': '한 달 좀 덜 된 기억들주머니에 넣은 채걷고 있어 몇 시간을혹시 몰라 네가 좋아했던코트를 입은 채나온 번화가 그때 마침찬바람 막아줄네가 이젠 없으니까추울 때 따스히안아줄 이가 없으니까친구들이 불러도나갈 수 없어 난창문 너머그저 바라봐 그때 마침눈이 오잖아우리 처음 만난 그 밤에도한참 동안 눈이 왔잖아지금 내가 생각나지않을 리가 없잖아눈이 오잖아그때 내가 밤하늘 내린하얀 눈 예쁘다 했잖아그랬잖아지금 눈이 오잖아Uh 눈이 오잖아이 말 핑계인 걸 알지만생각에 잠기곤 해거릴 거니는 내곁에 아무도 없을 때더 크게 들리네Oh 눈 내리는 소리이제 인정하자수많은 끝도 끝났어찬란한 그때의우린 이젠 없으니까돌아온 계절남아있는 건 없으니까추억을 지우는 게내 이별의 숙제창문 너머그저 바라봐 그때 마침눈이 오잖아우리 처음 만난 그 밤에도한참 동안 눈이 왔잖아지금 내가 생각나지않을 리가 없잖아눈이 오잖아그때 내가 밤하늘 내린하얀 눈 예쁘다 했잖아그랬잖아지금 눈이 오잖아마지막 안녕이 남은 거리에서 혼자너와의 약속들을 되뇌었어함께여야 할 너 없는 이 겨울의첫눈이 오잖아눈이 오지만우리 처음 만난 그 밤에도내렸던 그 눈이 오지만주머니 속 챙겨 나온 기억버려야만 해눈이 오지만우린 이대로의 모습으로남아있는 게 맞지만그래도지금그 눈이 오잖아'}
11 이제 나만 믿어요
{'곡명': '이제 나만 믿어요', '가수': '임영웅', '앨범': '내일은 미스터트롯 우승자 특전곡', '발매일': '2020.04.03', '장르': '성인가요', '좋아요': 146420, '가사': '무얼 믿은 걸까 부족했던 내게서나조차 못 믿던 내게 여태 머문 사람무얼 봤던 걸까 가진 것도 없던 내게무작정 내 손을 잡아 날 이끈 사람최고였어그대 눈 속에 비친 내 모습이제는 내

20 Next Level
{'곡명': 'Next Level', '가수': 'aespa', '앨범': 'Next Level', '발매일': '2021.05.17', '장르': '댄스', '좋아요': 235361, '가사': 'I’m on the Next Level Yeah절대적 룰을 지켜내 손을 놓지 말아결속은 나의 무기광야로 걸어가알아 네 Home ground위협에 맞서서제껴라 제껴라 제껴라상상도 못한 Black out유혹은 깊고 진해(Too hot too hot)맞잡은 손을 놓쳐난 절대 포기 못해I’m on the Next Level저 너머의 문을 열어Next Level널 결국엔 내가 부셔Next LevelKOSMO에 닿을 때까지Next Level제껴라 제껴라 제껴라La la la la la laLa la la la la laLa la la la la laLa la la la laI see the NU EVO.적대적인 고난과 슬픔은널 더 Popping 진화시켜That’s my naevis It’s my naevisYou lead, we follow감정들을 배운 다음Watch me while I make it outWatch me while I work it outWatch me while I make it outWatch me while I work it outWork it work it work it out감당할 수 없는 절망도내 믿음을 깨지 못해더 아픈 시련을 맞아도난 잡은 손을 놓지 않을게 Oh절대로 뒤를 돌아보지 말아광야의 것 탐내지 말아약속이 깨지면모두 걷잡을 수 없게 돼 언제부턴가 불안해져 가는 신호널 파괴하고 말 거야(We want it)Come on!Show me the way to KOSMO Yeah Black Mamba가 만들어낸 환각 퀘스트aespa, ae를 분리시켜놓길 원해 그래중심을 잃고 목소리도 잃고 비난받고사람들과 멀어지는 착각 속에naevis 우리 ae, ae들을 불러봐aespa의 Next Level “P.O.S”를 열어봐이건 R

30 끝사랑
{'곡명': '끝사랑', '가수': '임영웅', '앨범': '사랑의 콜센타 PART61', '발매일': '2021.06.29', '장르': '성인가요', '좋아요': 33318, '가사': '내가 이렇게 아픈데 그댄 어떨까요원래 떠나는 사람이 더 힘든 법인데아무 말하지 말아요 그대 마음 알아요간신히 참고 있는 날 울게 하지 마요이별은 시간을 멈추게 하니까모든걸 빼앗고 추억만 주니까아무리 웃어 보려고 안간힘 써 봐도밥 먹다가도 울겠지만그대 오직 그대만이내 첫사랑 내 끝사랑지금부터 달라질 수 없는 한 가지그대만이 영원한 내 사랑 wah woo wah그대도 나처럼 잘못했었다면그 곁에 머물기 수월했을까요사랑해 떠난다는 말 과분하다는 말코웃음 치던 나였지만그대 오직 그대만이내 첫사랑 내 끝사랑지금부터 그대 나를 잊고 살아도그대만이 영원한 내 사랑나는 다시는 사랑을 못할 것 같아요그대가 아니면'}
31 우리가 헤어져야 했던 이유
{'곡명': '우리가 헤어져야 했던 이유', '가수': '비비 (BIBI)', '앨범': '그 해 우리는 OST Part.2', '발매일': '2021.12.13', '장르': '발라드, 국내드라마', '좋아요': 41664, '가사': 'Maybe If IWoke up in the morningHearing your voiceMay If I was with you어쩌면 우리 어려웠던 날들함께 보냈었다면혹시 우린 어땠을까The distance between you and meIt never seems to disappear얼어붙은 나의 말들과 너만약에 우리 우연히다시 또 만난다면만약에 내가 널 위해조금 달라진다면우리가 헤어져야만 했던 이유도I would try try try안아줄 수 있을까조금은 다른기억의 조각들하지만 꼭 닮은그리움의 마음들은Only if youIf you comeThe distance between you and meIt never seems to disappear멈춰버린 우리 추억과 너만약에 우리 우연히다시 또 만

40 Child
{'곡명': 'Child', '가수': '마크 (MARK)', '앨범': 'Child - SM STATION : NCT LAB', '발매일': '2022.02.04', '장르': '랩/힙합', '좋아요': 47801, '가사': '너가 뭔데 내 마음을매달고 흔들어 날 어지럽게내가 문제아라면 왜항상 질문을 던지는 건데Oh I Oh I Oh I 서두르다가Oh I Oh I Oh I 서툴러지네I’m a child너가 바란대론 못 돼Have a good night자장가는 필요 없대It’s my question many questions사람들이 바라본 난 누구야Who am I who I’ll be다른 밤 같은 NightmareWhy why why kill my night너가 부숴버린 밤에내가 다칠 것 같아Blah blah blah y’all talk too muchDon’t waste my time거기까지만 해I pull at the strings when I’m pissed알 사람들만 알지 난 좀Twisted but the fittest to the point꿈이라는 게 태양이라면내 머리는 그걸빙빙 도는 Earth all dayI’m alert it’s a curse yeah it hurtsI don’t know my own strengthThere’s a lot that I don’t knowBut you learn when you don’t knowRed on my Valentino부숴도 부술래 내 책임감자유로울 때까지 뒤를 안 돌아봐꼭 먼 남이 되고 싶었어 이 사회와Propane이 부러워I need to blow up now투박해야 느껴지나 봐 이 맛가시를 길러 쉽게 못 넘기게 날How’d I get so bitterBrother liquorI don’t feel the same넌 어때 Tell me laterI’m a child너가 바란대론 못 돼Have a good night자장가는 필요 없대It’s my question many questio

...

55 Wake Up (Prod. 코드 쿤스트)
{'곡명': 'Wake Up (Prod. 코드 쿤스트)', '가수': '개코', '앨범': '쇼미더머니 10 Episode 1', '발매일': '2021.11.06', '장르': '랩/힙합', '좋아요': 78619, '가사': '"무거운 몸을 또 일으켜 세워난 부서지는 게 무섭지 않아힘들 땐 잠시만 갓길에 세워졸음쉼터는 찾으면 가까이 있어Wake up 해세상은 무에서 무유에서 full 소유그들은 여전히 굵은 선 긋기 중우린 무에서 유무에서 유그어진 투명한 선을 넘어Ready to move한때는 얻지 못했어 원한 걸계단 앞에서 무너지고 없었어 보람도굳게 먹었던 마음을 일으켜 다시 한번꾸겨진 지폐 몇 장이 다였어 쥔 거라곤차가웠던 이 서울 속 파도 같던 그 설움도밀려와 또 불만족난 믿지 않았어 거울도필요해 우린 큰 감동다신 뒤로 가지 않고 고개를 위로 불신하지 않고 무릅썼던 위험밤이고 낮이고 할 일 해 어떠한 것도 못 막지 옆으로 치워당하고 살기는 싫어 지저분했던 그 시절 기억들은 지워늦었지만 이젠 행복한 추억들만 머리 가득히 더해더는 아껴 먹지 마 엄마 내가 보탤 게 통장에다 더해내 팔 다리 생활의 달인30까지 무식하게 버텨 악바리하루 빨리 때려치란 아버지의 말이나오지 않게 벌떡 걷어차 이부자리날 끌어내리는 말들 선 넘어 반칙끄떡없어 반지하 출신의 간지잃을게 없는게 차라리 낫지그 몇 푼 가진다고 죽 쓰진 않지잘 깨고있어 내 한계넘어서 보여줘야 돼 반대가진 거 없이 여기 올라와 밤엔술 아니면 담배로 위로하고 봤네난 살았어 가난과 함께 근데오늘 부로 선 그을 거 같애이가 없다면 잇몸 써서라도만들어 성공은 내게서 탄생I wake up with no make up지겨웠던 악몽을 매일 꿨던과거를 뒤로 하고 위로 way up새로 산 조던을 신고 lay up어두웠던 만큼 밝은 내 아침 서러웠던 내 서울살이 마침고생한 끝에 낙이 찾아 왔지SINCE GO UP 이제 하늘에 닿지끼니때를 놓치며 지켜냈던 내 추임새정치와 정치질을 구분

63 사이렌 Remix (Feat. UNEDUCATED KID, Paul Blanco)
{'곡명': '사이렌 Remix (Feat. UNEDUCATED KID, Paul Blanco)', '가수': '호미들', '앨범': '사이렌 Remix', '발매일': '2021.03.20', '장르': '랩/힙합', '좋아요': 130256, '가사': '울려댔어 사이렌텅 빈 길거리엔도망치다 흘린 칼자루와 피가 흥건해우리 그때 어릴땐뭘 몰랐었지 man그냥 힘쎈 형이 제일로 멋졌었지 그땐그래 우린 살아나왔어 지옥이제 어딜가든 다 비옥수도 없이 맛본 치욕어릴때부터 입에 붙은 쌍욕절대 할 수 없었지 신고할 수 있는게 오직 기도어떻게 느끼겠어 피곤붉게 물들지 않을려 내 흰옷침대가 없어서 엄마랑 잤어매일 밤마다 우는 소리가 들려난 대체 뭘 해야 되는지 몰라서 그냥 계속했어 자는 척떨면서 BBQ 치킨을 시켰어엄마는 우리를 속였어고작 15000원 때문에엄마 내가 돈 어떻게든 벌어올게이제 남몰래 눈물 흘리지 않아도 돼내 몸 하나 불살라서라도 꺼내줄게난 그 모습 다시 실눈 뜨고 볼 순 없기에울려댔어 사이렌텅 빈 길거리엔도망치다 흘린 칼자루와 피가 흥건해우리 그때 어릴땐뭘 몰랐었지 man그냥 힘쎈 형이 제일로 멋졌었지 그땐그래 우린 살아나왔어 지옥이제 어딜가든 다 비옥수도 없이 맛본 치욕어릴때부터 입에 붙은 쌍욕절대 할 수 없었지 신고할 수 있는게 오직 기도어떻게 느끼겠어 피곤붉게 물들지 않을려 내 흰옷여기서 더 있다간 감방이 내 방종신형이 친구 책가방 안에어린 내가 짝사랑하던 그 여잔이 block에서 지금 몸 팔아새꺄 this real life김치 냄새 땜에 놀려대길래도시락통을 닫아That’s real life그런 날 보고 지 점심을 나눠준꼬마가 있었어That’s real life어느 날 관 속에그 친구의 얼굴을 만지게 됐어That’s울려댔어 사이렌텅 빈 길거리엔도망치다 흘린 칼자루와 피가 흥건해우리 그때 어릴땐뭘 몰랐었지 man그냥 힘쎈 형이 제일로 멋졌었지 그땐그래 우린 살아나왔어 

75 추억은 만남보다 이별에 남아
{'곡명': '추억은 만남보다 이별에 남아', '가수': '정동하', '앨범': '추억은 만남보다 이별에 남아', '발매일': '2021.01.31', '장르': '발라드', '좋아요': 77602, '가사': '아득한 시간 속에 아직우리 사랑이 남아있을까멀어지던 그날의 너를 따라 걸어도텅 빈 거리엔 미움만너의 흔적을 마주칠 때마다익숙함 속에 떠나보내던소중했던 모든 날들은후회로 남아이렇게 또 내게 다가오나 봐멍하니 혼자 남아나도 모르게 널 그리워하다떠오르는 네 생각에 하루를 살아선명했던 우리의추억은 만남보다 이별에 남아여전히 너를 사랑하게 하나 봐그날처럼저녁 빛에 밤이 물들면길게 늘어진 그림자처럼흩어질 듯 더 커져가는 너의 모습은아직도 내 안에 가득한가 봐멍하니 혼자 남아나도 모르게 널 그리워하다떠오르는 네 생각에 하루를 살아선명했던 우리의추억은 만남보다 이별에 남아여전히 너를사랑하고 있나 봐 바보처럼흩날리는 기억의 모든 순간마다 너를매일 이렇게 불러본다꼭 한 번쯤 너에게 해주고 싶었던 말너무 사랑했다고다시는 너를 떠나지 않겠다고여전히 너를 사랑하고 있다고그날 보다'}
76 Because
{'곡명': 'Because', '가수': 'ASH ISLAND', '앨범': 'Because', '발매일': '2022.01.25', '장르': '랩/힙합', '좋아요': 22724, '가사': 'Because i love u넌 몰랐겠지만우리가 끝이 날 것도알고 있었지만Still i love u넌 몰랐겠지만물론 더 이상 의미 없는 얘기야우린 끝을 향해 가고 있었고그런 모습마저 아름다웠어어느새 같이 있는 게너무 어색해서돌아서며 말했어우리 더는 마주하지 말자고우린 이미 알고 있었잖아너가 나를 밀고 집을 떠난 날우린 서로가 왜 그토록끝까지 애를 썼을까이런 건 의미 없다는 걸 알아그리워서 하는 얘기도 아니야그냥 그냥Because i love u넌 몰랐겠지만우리가 끝이 날 것도 알고 있었지만Still i love u넌 몰랐겠지만물론 더 이상 의미 없

84 검정색하트 (Feat. Leellamarz, BE'O)
{'곡명': "검정색하트 (Feat. Leellamarz, BE'O)", '가수': 'TOIL', '앨범': 'Between Sat & Sun', '발매일': '2022.01.30', '장르': '랩/힙합', '좋아요': 16361, '가사': '자꾸 잦아지는 다툼무뚝뚝해진 말투사랑한단 말을보내야 하니까 또 다른방법을 찾아 나는안아줄래 내 마음을평소완 조금 다른오늘은 검정색의 하트를 보내내가 소중 하단 말참 듣기 좋은 말들그 뒤에 다른 상황또 내게 주는 아픔들을모아서 쓰는 가사 듣기 좋니이 노래는 퍼지지 않기를 바래 길거리에사소한 감정 표현마저 사람들이 알잖아I don’t want that너는 뭘 원해나를 내버려 둬 실망하기 싫은데맞춰가는 건그래 한두번세번까지도 이해할게 사랑하는 걸I gotta move onto 다음번어차피 사랑이 그렇지 똑같은 걸역시 어려워나도 참 여전해서투르기만 한 감정 표현아주 조금그저 잠깐이면 돼반복일 뿐이잖아 우리 이대론자꾸 잦아지는 다툼무뚝뚝해진 말투사랑한단 말을보내야 하니까 또 다른방법을 찾아 나는안아줄래 내 마음을평소완 조금 다른오늘은 검정색의 하트를 보내오늘은 검정색의 하트를 보내오늘은 검정색의 하트를 보내오늘은 검정색의 하트를 보내오늘은 검정색의 하트를 보내티 내지 마 baby대체 누군데 쟤넨뭣도 아닌 고마운 검정 하트에 말싸움해아 진짜 왜 이래나는 안돼 이젠내 맘은 채이고 채였어지금 say thatLove 그딴 게 뭔데자꾸 넌 내게 하라잖아 선택예상했어 이따위 뻔한 전개낌새가 보였거든 알아 넌 매일그래도 넌 왜 끝까지 뻔해약속 한마디도 없네그래 나 서운해라고말하기조차 무서워하며 이제 난Black heartBlack heart너는 likeBlack holeBlack hole같이 날빨아들이지만사라지게 하니까혼자인 어두운 밤을당연시하게 만들잖아넌Black heartBlack heart너는 likeBlack holeBlack hole같이 날빨아들이지만사라지게 하니까혼자인

93 멜로디
{'곡명': '멜로디', '가수': 'ASH ISLAND', '앨범': '멜로디', '발매일': '2021.02.25', '장르': '랩/힙합', '좋아요': 125376, '가사': "어딘가 들어본 듯한 이 Melody떠올라 작은 기억들이 My memories날 담은 소파는 커 It's too much Too much기억할수록 난 되어만 가 Loser Loser어딘가 들어본 듯한 이 Melody떠올라 작은 기억들이 My memories날 담은 소파는 커 It's too much Too much기억할수록 난 되어만 가 Loser LoserI'm a Loser 난 처음 너의 바람이 되고 싶어 세상을 다 주겠다고 약속을 해많은 눈들의 착각이나에겐 혼란이 되어 너만 힘들게 했지더 잃을 것도 없었는데날 너와 비교했어 참 나는 못된사람이 됐고 모든 내 말들은널 위한 거였단 핑계가 돼근데 이젠 혼자 살기에는버거운 집과 혼자 앉아있긴너무 커 소파또 다 쓰지도 못하는 돈과 옷널 데려다줄 만한 큰 차뭐 다 가져버리고 나서 난TV에 너와 함께 보던영화 흘러나오다가혼잣말 옆에 없잖아 너가하루 종일 싸우고 안은 채잠들어도 좋아다 괜찮으니 와서 채워주라 이 소파혼자는 이 공간이 너무 커 필요해 너가필요해 너가어딘가 들어본 듯한 이 Melody떠올라 작은 기억들이 My memories날 담은 소파는 커 It's too much Too much기억할수록 난 되어만 가 Loser Loser어딘가 들어본 듯한 이 Melody떠올라 작은 기억들이 My memories날 담은 소파는 커 It’s too muchToo much기억할수록 난 되어만 가 Loser LoserI'm a Loser Ooh 영원할 것 같았던 약속들손을 잡아주겠다던 너와 나 둘앞이 안 보일 때 너가 돼준 내 눈겨울에 함께 봤던 눈불타올랐지 아무리 추워도 난With you불이 꺼져 재만 남아도 Lovin u라고 말하던 때 그때 그 기억인 듯아직도 너와 듣던 노래들의 가사들 뻔한듯했던 그 Melody가 익숙하던 예

### json 파일로 저장

In [41]:
import json

with open('data/songs.json', 'w', encoding='utf-8') as file:
    json.dump(song_detail_list, file)
    
with open('data/songs.json', 'r', encoding='utf-8') as file:
    song_json = json.loads(file.read())

# print(song_json)

### Pandas의 DataFrame 객체 사용하기

In [45]:
# song_detail_list를 읽어서 DataFrame 객체를 생성하는 방법
import pandas as pd

song_df2 = pd.DataFrame(columns=['곡명','가수','앨범','발매일','장르','좋아요','가사'])

# 1개의 행 = Series 객체 / 1개의 열 = Series 객체
for song_detail in song_detail_list:
    series_obj = pd.Series(song_dict)
    song_df2 = song_df2.append(se)

,곡명,가수,앨범,발매일,장르,좋아요,가사


In [44]:
import pandas as pd

song_df = pd.read_json('data/songs.json')
song_df
song_df.head() # 앞에 5개만 출력
song_df.tail() # 뒤에 5개만 출력

,곡명,가수,앨범,발매일,장르,좋아요,가사
95,좋아좋아,조정석,슬기로운 의사생활 시즌2 OST Part 5,2021.07.16,"발라드, 국내드라마",83249,처음 널 만나는 날노란 세 송이 장미를 들고룰루랄라 신촌을 향하는내 가슴은 마냥 두...
96,나 그댈위해 시 한편을 쓰겠어,케이시 (Kassy),옛이야기,2021.10.28,발라드,31439,나 그댈 위해 시 한 편을 쓰겠어나 그대에게 못다 한 얘기 많아차마 그때 하지 못 ...
97,깨우지 마,MC몽,X by X [ 꿈 ],2022.01.21,랩/힙합,7703,깨우지 마 꿈속으로도망치듯 잠에 든 날하루 중 내가 유일하게무책임해도 되는 시간날 ...
98,Still I Love You,토요태,Still I Love You,2021.12.18,R&B/Soul,31471,여보세요어 잘 지내지오빠 전화하지 말랬잖아아 그치 어우 야 밖에 많이 춥다따뜻하게 ...
99,MBTI (Feat. 쿠기 & 로꼬) (Prod. GRAY),BE'O (비오),쇼미더머니 10 Semi Final,2021.11.27,랩/힙합,45093,Million 내 몸값을 더 올려Billion 내 몸값을 더 올려Trillion 내...
